In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
%matplotlib inline

In [2]:
import datetime
# import cPickle as pickle
import pickle
import csv
import numpy as np
import random
import sys
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

### load voca

In [3]:
# dic_voca = ''
# with open('../data/nela-18/whole/dic_mincutN.pkl', 'rb') as f:
#     dic_voca = pickle.load(f)

dic_voca = ''
with open('../data/nela-18-past-method/whole/dic_mincutN.pkl', 'rb') as f:
    dic_voca = pickle.load(f)

#### for data processing

In [4]:
import copy
dic_voca_lower = copy.deepcopy(dic_voca)

In [5]:
dic_voca_lower['<eos>'] = dic_voca_lower['<EOS>']
dic_voca_lower['<eop>'] = dic_voca_lower['<EOP>']

del dic_voca_lower['<EOS>']
del dic_voca_lower['<EOP>']

In [6]:
len(dic_voca_lower)

155544

In [7]:
print(dic_voca_lower[''], dic_voca_lower['<UNK>'], dic_voca_lower['<eos>'], dic_voca_lower['<eop>'])

0 1 2 3


## stats

In [8]:
# import csv
# import sys
# import numpy as np

# data= []
# with open('../data/raw/real_world_articles_ascii.tsv', 'r') as f:
#     data_csv = csv.reader(f, delimiter='\t')
#     for row in data_csv:
#         data.append(row)

In [9]:
# def print_info(data):
#     print("mean", np.average(data))
#     print("std", np.std(data))
#     print("max", np.max(data))
#     print("95.xx coverage", np.average(data) +  2*np.std(data) )
#     print("99.73 coverage", np.average(data) +  3*np.std(data) )
#     print("99.95 coverage", np.average(data) +  3.5*np.std(data) )
#     print("99.99 coverage", np.average(data) +  4*np.std(data) )

In [10]:
# head = [x[1].strip() for x in data]
# head_len = [len(x.split()) for x in head]
# print('head_len')
# print_info(head_len)

In [11]:
# body = [x[2].strip() for x in data]

In [12]:
# body_len = [len(x.split()) for x in body ]
# print('body_len')
# print_info(body_len)

In [13]:
# context_len = [len(x.split('<EOP>')) for x in body]
# print('context_len')
# print_info(context_len)

In [14]:
# body_sentence = []
# for sent in body:
#     sent = sent.split('<EOP>')
#     body_sentence.extend(sent)
# body_len = [ len(x.split()) for x in body_sentence ]    
# print('body_sentence')
# print_info(body_len)

# encode to numpy

In [15]:
def fit_length(data, max_len_t, max_len_b):
    data_t, data_b = data
    
    list_zeros = np.zeros(max_len_b, 'int32').tolist()
    fl_data_t = []
    for datum in data_t:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_t:
            fl_data_t.append( datum[:max_len_t] )
        else:
            fl_data_t.append( datum + list_zeros[:(max_len_t-_len)] )
            
    fl_data_b = []
    for datum in data_b:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_b:
            fl_data_b.append( datum[:max_len_b] )
        else:
            fl_data_b.append( datum + list_zeros[:(max_len_b-_len)] )
    
    np_data_t = np.asarray(fl_data_t, dtype='int32')
    np_data_b = np.asarray(fl_data_b, dtype='int32')
    
    data = [np_data_t, np_data_b]
    return data

In [16]:
csv_reader = csv.reader(open('../data/raw/real_world_articles_ascii.tsv', 'r'), delimiter='\t')

print (datetime.datetime.now().isoformat())
ids = []
heads = []
bodys = []
labels = []
for n, row in enumerate(csv_reader):
    if (n+1) % 10000 == 0: print (n+1),
    
    ids.append(row[0])
    labels.append(int(row[3]))
    
    head = []
    for tkn in row[1].lower().strip().split():
        if tkn in dic_voca_lower:
            head.append(dic_voca_lower[tkn])
        else:
            head.append(1)
            
    heads.append(head)
    
    body = []
    for tkn in row[2].lower().strip().split():
        if tkn in dic_voca_lower:
            body.append(dic_voca_lower[tkn])
        else:
            body.append(1)
            
    bodys.append(body)
    
print (n+1, 'Done')
print (datetime.datetime.now().isoformat()) # ~5 mins

2019-12-05T10:13:24.369422
10000
20000
30000
40000
50000
60000
70000
80000
90000
95069 Done
2019-12-05T10:13:40.549447


In [17]:
print (datetime.datetime.now().isoformat())
[np_heads, np_bodys] = fit_length([heads, bodys], 25, 2800)
print (datetime.datetime.now().isoformat()) # ~3 mins

2019-12-05T10:13:40.554698
2019-12-05T10:14:02.634130


In [19]:
print (datetime.datetime.now().isoformat())
t_trainpath = '../data/nela-18-past-method/whole/test/test_title.npy'
np.save(t_trainpath, np_heads)
b_trainpath = '../data/nela-18-past-method/whole/test/test_body.npy'
np.save(b_trainpath, np_bodys)
l_trainpath = '../data/nela-18-past-method/whole/test/test_label.npy'
np.save(l_trainpath, labels)
print (datetime.datetime.now().isoformat())

2019-12-05T10:14:32.151488
2019-12-05T10:14:32.825417
